In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
import collections
import time
import random

## 3.0 序论

- `dict` 类型是 Python 语言的基石， 跟其有关的内置函数均在 `__builtins__.__dict__` 模块中
- 散列表是字典性能出众的根本原因
- 集合 (set) 的实现也依赖于散列表

## 3.1 泛映射类型

- `collections.abc` 模块中有 `Mapping` 和 `MutableMapping` 两个抽象基类，它们为 `dict` 和其他类似的类型定义了形式接口
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200302160234.png width=600>
  - 抽象基类的直接作用是作为形式化的文档
  - 非抽象的映射类型一般不会直接继承抽象基类，而是直接对 `dict` 或 `collections.UserDIict` 进行扩展
  - 可以用 `isinstance` 来判定某个数据是不是广义的映射类型

In [0]:
my_dict = {}
isinstance(my_dict, collections.abc.Mapping)

True

- 标准库里的所有映射类型都是用 `dict` 来实现的， 因此它们有一个共同的限制，即只有*可散列* 的数据类型可以用做这些映射里的键

##### 可散列数据类型

- [定义](https://docs.python.org/3/glossary.html#term-hashable)
  - 一个可散列对象，在其生命周期内，其散列值不会发生改变
  - 可散列对象需要实现 `__hash__()` 方法
  - 为了能与其它键做比较，可散列对象还需要实现 `__eq__()` 方法
    - 如果两个可散列对象是相等的，则它们的散列值一定是一样的

- 原子不可变数据类型（`str`, `bytes` 和数值类型） 都是可散列类型
- `frozenset` 里只能容下可散列类型，因此其也是可散列的

###### 只有当一个元组所包含的都是可散列数据类型的情况下，其才是可散列的

In [0]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [0]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: ignored

In [0]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

###### 用户自定义数据类型

- 一般来说用户自定义的类型对象是可散列的，散列值即是它的 `id()` 函数的返回值
- 如果一个对象实现了 `__eq__` 方法，并在方法中用到了这个对象的内部状态的话，则只有当这些内部状态都是不可变的情况下，这个对象才是可散列的

##### 字典的构造方法

In [0]:
a = dict(one=1, two=2, three=3)
b = {'one':1, 'two':2, 'three':3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2} )
a == b == c == d == e

True

## 3.2 字典推导

- 字典可以从任何以键值对作为元素的可迭代对象中构告出字典

###### 示例 3-1 字典推导的应用

In [0]:
DIAL_CODES = [
(86, 'China'),
(91, 'India'),
(1, 'United States'), 
(62, 'Indonesia'),
(55, 'Brazil'), 
(92, 'Pakistan'), 
(880, 'Bangladesh'), 
(234, 'Nigeria' ), 
(7,  'Russia'), 
(81, 'Japan') ]

In [0]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [0]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 7: 'RUSSIA', 55: 'BRAZIL', 62: 'INDONESIA'}

## 3.3 常见的映射方法

###### 表 3-1 `dcit`、 `collections.defaultdict` 和 `collections.OrderedDict` 三种映射类型的方法列表

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200302185739.png width=800>

- `update` 处理参数 `m` 的方式，为典型的 ‘鸭子类型’
  - 首先检查 `m` 中是否有 `keys` 方法，如果有，则将其当作映射对象来处理 
  - 否则，函会退一步，将 `m` 当做包含了键值对 `(key, value)` 的迭代器

##### 用 `setdefalut` 处理找不到的键

- `setdefalut` 方法可以用来更新字典中存放的可变值（如列表）， 从而避免重复的键搜索

###### 示例 3-2 获取单词出现的频率

In [0]:
%%writefile ch3/index0.py
"""创建从一个单词到其出现情况的映射"""

import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}
with open(sys.argv[1]) as fp:
  for line_no, line in enumerate(fp, 1):
    for match in WORD_RE.finditer(line):
      word = match.group()
      column_no = match.start() + 1
      location = (line_no, column_no)
      occurences = index.get(word, [])  # get 只会在当值不存在时，返回默认值
      occurences.append(location)  
      index[word] = occurences  # 应对值不存在的情况，又涉及一次查询

# 以字母序打印输出结果
for word in sorted(index, key=str.upper):
  print(word, index[word])


Overwriting ch3/index0.py


In [0]:
!python ch3/index0.py ch3/zen.txt

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

###### 示例 3-4：  用 `dict.setdefault` 来改进示例 3-2

In [0]:
%%writefile ch3/index.py
"""创建从一个单词到其出现情况的映射"""

import sys
import re

WORD_RE = re.compile("\w+")

index = {}
with open(sys.argv[1]) as fp:
  for line_no, line in enumerate(fp, 1):
    for match in WORD_RE.finditer(line):
      word = match.group()
      column_no = match.start() + 1
      location = (line_no, column_no)
      index.setdefault(word, []).append(location) # 一次查询即可达到目的，因为如果键不存在， d. setdefalut 会令 d[k] = default，并返回 default

# 以字母序打印出结果
for word in sorted(index, key=str.upper):
  print(word, index[word])
      

Writing ch3/index.py


In [0]:
!python ch3/index.py ch3/zen.txt

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

## 3.4 映射的弹性键查询

- 如果一个键在映射中不存在，也希望从这个键读取值时能读到默认值，有两种方法
  1. 使用 `collections.defaultdict` 类
  2. 定义自己的子类，然后在子类中实现 `__misssing__` 方法

### 3.4.1 `defaultdict`：处理找不到的键的一个选择

- 设 `dd = collections.defaultdict(list)`， 如果 `new-key` 在 `dd` 中不存在，则 `dd['new-key']`
会执行以下操作
  1. 调用 `list()` 来建立一个新列表
  2. 将这个新列表作为值， `new-key` 作为它的键， 放到 `dd` 中
  3. 返回这个值的引用
- 用来生成默认值的可迭代对象存放在名为 `default_factory` 的实例
- 实现这一功能的方法为 `__missing__`，其会在 `default_dict` 找不到键的时候调用 `default_factory`
  - `default_factory` 只会在 `__getitem__` 方法中被调用，在其它方法中则不会
    - `dd[k]` 如果 `k` 不在字典中， 则会调用 `default_factory`
    - `dd.get(k)`，如果 `k` 不在典中，则会返回 `None`

###### 示例 3-5 index_default.py: 利用 `defaultdict` 实例而不是 `setdefault` 方法

In [0]:
%%writefile ch3/index_default.py
"""创建一个从单词到其出现情况的映射"""

import sys
import re
import collections

WORD_RE = re.compile("\w+")

index = collections.defaultdict(list)

with open(sys.argv[1]) as fp:
  for line_no, line in enumerate(fp, 1):
    for match in WORD_RE.finditer(line):
      word = match.group()
      column_no = match.start() + 1
      location = (line_no, column_no)
      index[word].append(location)

# 按字母序打印结果
for word in sorted(index, key=str.upper):
  print(word, index[word])


Overwriting ch3/index_default.py


In [0]:
!python ch3/index_default.py ch3/zen.txt

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]


### 3.4.2 特殊方法 `__missing__`

- 所有映射类型在处理找不到的键时，都有牵扯到 `__missing__` 方法
- 当 `__getitem__` 碰到找不到的键时，其会自动的调用 `__missing__` 方法
- `__missing__` 对于 `get` 或者 `__contains__` 这些方法的使用没有影响

###### 视图

- [Dictionary view objects](https://docs.python.org/3/library/stdtypes.html#dictionary-view-objects)
- `my_dict.keys()` 返回的是一个视图，其就像一个集合，而且跟字典类似，在视图里查找一个元素的速度很快

###### 示例 3-7  `StrKeyDict0` 在查询时把非字符串的键转换为字符串

- 并没有限制 `StrKeyDict0` 传入的键必须为字符串，从而让查找更加友好

In [0]:
class StrKeyDict0(dict):
  def __missing__(self, key):
    if isinstance(key, str):  # 为了防止键不存在时，陷入无限次递归调用
      raise KeyError(key)
    return self[str(key)]
  
  def get(self, key, default=None):
    try:
      return self[key]
    except KeyError:
      return default
  
  def __contains__(self, key):
    return key in self.keys() or str(key) in self.keys()  # 不能使用 key in my_dict 的方式，不然会陷入无限循环 
  


In [0]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [0]:
d[4]

'four'

In [0]:
d[1]

KeyError: ignored

In [0]:
d.get('2')

'two'

In [0]:
d.get(4)

'four'

In [0]:
d.get(1, 'N/A')

'N/A'

In [0]:
2 in d

True

In [0]:
1 in d

False

## 3.5 字典的变种

#### `collections.OrderedDict`

- 在添加键时会保持顺序，因此键的迭代次序是一致的
- `OrderedDict` 的 `popitem` 方法默认删除并返回的是字典中的最后一个元素
  - `my_odict.popitem(last=False)` 其删除并返回的是第一个被添加进去的元素

#### `collections.ChainMap`

- 此类型可以容纳数个不同的映射对象，然后在进行键查找操作时，这些对象会被当做一个整体逐一查找
- 这个功能在给有嵌套作用域的语言做解释的时候很有用

In [0]:
import builtins
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))

#### `collections.Counter`

- 此映射类型会给键准备一个整数计数器，每次更新一个键时即会增加这个计数器
  - 即此类型可以用来给可散列对象计数
  - `Counter` 实现了 `+` 和 `-` 运算符来合并记录
  - `Counter` 还支持 `most_common([n])` 这类有用的方法，其会按次序返回映射里最常见的 `n` 个键和它们
  的计数

###### 利用 `Counter` 来计算单词中各个字母出现的次数

In [0]:
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [0]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'c': 1, 'd': 1, 'r': 2, 'z': 3})

In [0]:
ct.most_common(2)

[('a', 10), ('z', 3)]

#### collections.UserDict

- 此类就是把标准的 `dict` 用纯 Python 又实现了一遍
- 主要是用来让用户继承写子类的

## 3.6 子类化 `UserDict`

- 以 `UserDict` 为基类，比普通的 `dict` 为基类来得方便
  - 因为 `dict` 会某些方法的实现上走一些捷径，导致不得不在字类中重写这些方法
  - `UserDict` 并不是 `dict` 的子类，但 `UserDict` 中有一个属性 `data`， 其是 `dict` 的实例
    - 如此重新改写 `__setitem__` 即可避免出现示例 3-7 中可能出现的递归调用的情况

###### 示例 3-8 无论添加、更新还是查询操作，`StrKeyDict`都将非字符串转换为字符串

In [0]:
class StrKeyDict(collections.UserDict):
  def __setitem__(self, key, item):
    self.data[key] = item
  
  def __missing__(self, key):
    if isinstance(key, str):
      raise KeyError(key)
    return self[str(key)]
  
  def __contains__(self, key):
    return str(key) in self.data
    

In [0]:
d = StrKeyDict([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [0]:
d[4]

'four'

In [0]:
d[1]

KeyError: ignored

In [0]:
d.get('2')

'two'

In [0]:
d.get(4)

'four'

In [0]:
d.get(1, 'N/A')

'N/A'

In [0]:
2 in d

True

In [0]:
1 in d

False

#### 抽象基类提供的方法

- `UserDict` 继承的是 `MutableMapping`，因此 `StrKeyDict` 中剩下的映射类型方法都是从 `UserDict`， 
`MutalbeMapping` 和 `Mapping` 中这些超类中继承得到的

##### `MutableMapping.update`

- 此参数可直接被利用，而且其还存在于 `__init__` 中， 让构造方法可以利用传入的各种参数来新建实例
- 其背后是在用 `self[key] = value` 来添加新值的，其实是在调用 `__setitem__` 方法

##### `Mapping.get`

- 有 `StrDictKey0` 中，需要改写 `get` 方法，以便其表现的与 `__getitem__` 一致
- 在 `StrDictKey` 中，其继承了 `Mapping.get` 方法，而其原码与 `StrDictKey0` 中的 `get` 一模一样

## 3.7 不可变映射类型

- 其目的是为了防止用户错误的修改某个映射

- `types` 模块中的 `MappingProxyType` 类，如果给其一个映射，则其会返回一个动态的只读的映射视图
  - 动态指得是如果对原映射做出了改动，则可以通过这个视图观察到
  - 只读指得是无法通过这个视图对原映射作出修改

###### 示例 3-9 用 `MappingProxyType` 来获取字典的只读实例 `mappingproxy`

In [0]:
from types import MappingProxyType
d = {1:'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [0]:
d_proxy[1]

'A'

In [0]:
d_proxy[2] = 'x'

TypeError: ignored

In [0]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [0]:
d_proxy[2]

'B'

## 3.8 集合论

- 集合的本质是许多唯一对象的聚集。因此，集合可用于去重

In [0]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)

{'eggs', 'spam'}

In [0]:
list(set(l))

['spam', 'eggs']

- 集合中的元素必须是可散列的， `set` 类本身却是不可散列的，但是 `frozenset` 可以

- 集合支持很多中缀运算符，设 `a`，`b` 为两个集合， 则
  - `a|b` 返回合集
  - `a&b` 返回交集
  - `a-b` 返㚙差集
回的是差集
  1. 合理利用中缀运算符，可以使代码的行数变少，减少 Pyhon 程序的运行时间
  2. 可以让代码更易读，从而更容易判断程序的正确性

###### 示例 3-10  `neddles` 元素在 `haystack` 中出现的次数，两个变量均为 `set` 类型

```python
found = len(neddles & haystack)
```
- 如果不使用交集操作，则代码会变成示例 3-11

###### 示例 3-11 `neddles` 元素在 `haystack` 中出现的次数 

```python
found = 0
for n in neddles:
  if n in haystack:
    found += 1
```

- 比示例 3-10 较慢，但是可以用于任何可迭代对象的 `neddles` 和 `haystack` 中
- 可以将可迭代对象转换为集合，然后再使用交集操作，如示例 3-12

###### 使用交集操作，但对象可以是任何可迭代对象

```python
found = len(set(neddles) & set(haystack))
# 另一种写法
found = len(set(neddles).intersection(haystack))  # intersection 会将 haystack 自动转换为 set 类型
```

### 3.8.1 集合字面量

- 集合的字面量 ---- `{1}, {1, 2}`
- 如果集合是空集，则必须写为 set() 的形式，`{}` 创建是空字典

In [0]:
s = {1}
type(s)

set

In [0]:
s

{1}

In [0]:
s.pop()

1

In [0]:
s

set()

- `frozenset` 没有特殊的字面量句法，只能采用构造方法

In [0]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

###### 字面量句法 `{1,2,3}` 比 构造方法 `set([1, 2, 3])` 更快

In [0]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [0]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


- `set([1])` 会先查找构告函数 `set`，然后新建一个列表，之后将这个列表传递给 `set`
- `{1}` 会直接调用 `BUILD_SET` 字节码完成构造工作

### 3.8.2 集合推导

###### 示例 3-13 新建一个字符集合，该集合中每个字符的 `Unicode` 名字中都有 `SIGN` 这个单词

In [0]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### 3.8.3 集合的操作

- `set` 的 UML 类图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200303095331.png width=600>

###### 表 3-2 集合的数学运算

- 中缀运算符要求两侧的被操作数均为集合类型，其他的所有方法只要求传入的参数是可迭代对象

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200303173156.png width=800> 

###### 表 3-3： 集合的比较运算符，返回的是布尔类型

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200303173312.png width=800>

###### 表 3-4： 集合类型的其他方法

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200303173348.png width=800>

## 3.9 `dict` 和 `set` 的背后

### 3.9.1 一个关于效率的实验

- `haystack` 中有 1000 万个不同的双精度浮点数， `neddles` 中含有 1000 个不同的浮点数，其中 500 在 `haystack` 中，
500 个肯定不在 `haystack` 中

##### 代码实现

###### container_perftest_datagen.py

In [0]:
%%writefile ch3/container_perftest_datagen.py

"""
Generate data for container performance test
"""

import random
import array

MAX_EXPONENT = 7
HAYSTACK_LEN = 10 ** MAX_EXPONENT
NEEDLES_LEN = 10 ** (MAX_EXPONENT - 1)
SAMPLE_LEN = HAYSTACK_LEN + NEEDLES_LEN // 2

needles = array.array('d')

sample = {1/random.random() for i in range(SAMPLE_LEN)}
print('initial sample: %d elements' % len(sample))

# complete sample, in case duplicate random numbers were discarded
while len(sample) < SAMPLE_LEN:
    sample.add(1/random.random())

print('complete sample: %d elements' % len(sample))

sample = array.array('d', sample)
random.shuffle(sample)

not_selected = sample[:NEEDLES_LEN // 2]
print('not selected: %d samples' % len(not_selected))
print('  writing not_selected.arr')
with open('ch3/not_selected.arr', 'wb') as fp:
    not_selected.tofile(fp)

selected = sample[NEEDLES_LEN // 2:]
print('selected: %d samples' % len(selected))
print('  writing selected.arr')
with open('ch3/selected.arr', 'wb') as fp:
    selected.tofile(fp)

Overwriting ch3/container_perftest_datagen.py


###### container_perftest.py

In [0]:
%%writefile ch3/container_perftest.py
"""
Container ``in`` operator performance test
"""
import sys
import timeit

SETUP = '''
import array
selected = array.array('d')
with open('ch3/selected.arr', 'rb') as fp:
    selected.fromfile(fp, {size})
if {container_type} is dict:
    haystack = dict.fromkeys(selected, 1)
else:
    haystack = {container_type}(selected)
if {verbose}:
    print(type(haystack), end='  ')
    print('haystack: %10d' % len(haystack), end='  ')
needles = array.array('d')
with open('ch3/not_selected.arr', 'rb') as fp:
    needles.fromfile(fp, 500)
needles.extend(selected[::{size}//500])
if {verbose}:
    print(' needles: %10d' % len(needles), end='  ')
'''

TEST = '''
found = 0
for n in needles:
    if n in haystack:
        found += 1
if {verbose}:
    print('  found: %10d' % found)
'''

SETUP1 = SETUP + """

needles = set(needles)
"""

TEST1 = '''
found = len(needles & haystack)
if {verbose}:
    print('  found: %10d' % found)
'''

def test(container_type, verbose):
    MAX_EXPONENT = 7
    for n in range(3, MAX_EXPONENT + 1):
        size = 10**n
        if container_type == 'set_intersection':
          setup = SETUP1.format(container_type='set',
                             size=size, verbose=verbose)
          test = TEST1.format(verbose=verbose)
        else:
          setup = SETUP.format(container_type=container_type,
                             size=size, verbose=verbose)
          test = TEST.format(verbose=verbose)
        tt = timeit.repeat(stmt=test, setup=setup, repeat=5, number=1)
        print('|{:{}d}|{:f}'.format(size, MAX_EXPONENT + 1, min(tt)))

if __name__=='__main__':
    if '-v' in sys.argv:
        sys.argv.remove('-v')
        verbose = True
    else:
        verbose = False
    if len(sys.argv) != 2:
        print('Usage: %s <container_type>' % sys.argv[0])
    else:
        test(sys.argv[1], verbose)

Overwriting ch3/container_perftest.py


###### 测试

In [0]:
!python ch3/container_perftest_datagen.py

initial sample: 10500000 elements
complete sample: 10500000 elements
not selected: 500000 samples
  writing not_selected.arr
selected: 10000000 samples
  writing selected.arr


In [0]:
!python ch3/container_perftest.py list

|    1000|0.007193
|   10000|0.070712
|  100000|0.709263
| 1000000|7.171768
|10000000|72.314867


In [0]:
!python ch3/container_perftest.py dict

|    1000|0.000139
|   10000|0.000172
|  100000|0.000215
| 1000000|0.000535
|10000000|0.000671


In [0]:
!python ch3/container_perftest.py set

|    1000|0.000174
|   10000|0.000190
|  100000|0.000163
| 1000000|0.000369
|10000000|0.000451


In [0]:
!python ch3/container_perftest.py set_intersection

|    1000|0.000085
|   10000|0.000103
|  100000|0.000154
| 1000000|0.000322
|10000000|0.000449


### 3.9.2 字典中的散列表

- Python 的 `dict` 类型是通过散列表来实现的
- 散列表的实质是一个稀疏数组（总是有空白元素的数组称为稀疏数组），散列表中的单元叫做表元(bucket)
- 在 dict 的散列表中，每个键值对占用一个表元，每个表元有两个部分，一是对键的引用，另一个是对值的引用
- 表元的大小一致，所以可以通过偏移量来读取某个表元
- python 会设法保证有三分之一的表元是空的，在快达到这个阀值时，原有的散列表会被复制到一个更大的空间中

#### 1. 散列值和相等性

- 如果两个对象在比较时是相等的，则它们的散列值必须相等，否则散列表就不能正常工作
- 散列值在索引空间中应尽量分散开，因此越是相似但不相等的对象，它们的散列值差别应该越大

###### 示例 3-16 在 32 位的 Python 中， 1、 1.0001、 1.0002 和 1.0003 几个数的散列值

In [118]:
%%writefile ch3/hash_diff.py
import sys

MAX_BITS = len(format(sys.maxsize, 'b'))
print('%s-bit Python build' % (MAX_BITS + 1))

def hash_diff(o1, o2):
    h1 = '{:0>{}b}'.format(hash(o1), MAX_BITS)
    h2 = '{:0>{}b}'.format(hash(o2), MAX_BITS)
    diff = ''.join('!' if b1 != b2 else ' ' for b1, b2 in zip(h1, h2))
    count = '!= {}'.format(diff.count('!'))
    width = max(len(repr(o1)), len(repr(o2)), 8)
    sep = '-' * (width * 2 + MAX_BITS)
    return '{!r:{width}} {}\n{:{width}} {} {}\n{!r:{width}} {}\n{}'.format(
    		o1, h1, ' ' * width, diff, count, o2, h2, sep, width=width)

if __name__ == '__main__':
    print(hash_diff(1, 1.0))
    print(hash_diff(1.0, 1.0001))
    print(hash_diff(1.0001, 1.0002))
    print(hash_diff(1.0002, 1.0003))


Overwriting ch3/hash_diff.py


In [127]:
!python ch3/hash_diff.py

64-bit Python build
1        000000000000000000000000000000000000000000000000000000000000001
                                                                         != 0
1.0      000000000000000000000000000000000000000000000000000000000000001
-------------------------------------------------------------------------------
1.0      000000000000000000000000000000000000000000000000000000000000001
                        !! !   !! !! !!!   ! !!! ! !!   !!!   !          != 21
1.0001   000000000000000110100011011011100010111010110001110001000000001
-------------------------------------------------------------------------------
1.0001   000000000000000110100011011011100010111010110001110001000000001
                       ! !!!  ! !! !!  !  !!!  !!!! !  !  !  !!          != 22
1.0002   000000000000001101000110110111000101110101100011100010000000001
-------------------------------------------------------------------------------
1.0002   000000000000001101000110110111000101110101100011100010000

###### 随机加盐

- `str`, `bytes` 和 `datatime` 对象的散列值计算过程中多了随机‘加盐’的过程
- 所加盐值是 Python 进程内的一个常量，但每次启动 Python 解释器都会生成一个新的盐值
- 随机盐值的加入是为了防止 DOS 攻击

#### 2. 散列表算法

- 从字典中取键的流程图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200303195824.png width=700>
- 添加或更新键值的操作与从字典中取键基本相同
  - 添加键值时，在发现空的表元时会放入一个元素
  - 更新键值时，在找到相应的表元后，原表中的值会被替换为新的值
- 插入新值
  - 插入新值时， Python 会按照散列表的拥挤程度来决定是否重新分配内存为其重新扩容
  - 如果增加了散列表的大小，则散列值所占的位数和用做索引的位数会随之增加
  - 如此做的目的是为了减小散列冲突的概率

### 3.9.3 `dict` 的实现及其导致的结果

#### 1. 键必须是可散列的

#### 2. 字典在内存中的开销巨大

#### 3. 键查询很快

- `dict` 的实现是典型的空间换时间
  - 字典类型有着巨大的内存开销
  - 但提供了无视数据量大小的快速访问 -- 只要字典能被装入内存中

#### 4. 键的次序取决于添加顺序

- 当两个键发生冲突时，键在字典中出现的顺序与键的添加顺序有关

###### 示例 3-17 将同样的数据以不同的顺序添加到 3 个字典里

In [128]:
DIAL_CODES = [
(86, 'China'),
(91, 'India'),
(1, 'United States'), 
(62, 'Indonesia'),
(55, 'Brazil'), 
(92, 'Pakistan'), 
(880, 'Bangladesh'), 
(234, 'Nigeria' ), 
(7,  'Russia'), 
(81, 'Japan') ]

d1 = dict(DIAL_CODES)
print('d1:', d1.keys())
d2 = dict(sorted(DIAL_CODES))
print('d2:', d2.keys())
d3 = dict(sorted(DIAL_CODES, key=lambda x: x[1]))
print('d3:', d3.keys())

d1: dict_keys([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])
d2: dict_keys([1, 7, 55, 62, 81, 86, 91, 92, 234, 880])
d3: dict_keys([880, 55, 86, 91, 62, 81, 234, 92, 7, 1])


#### 5. 往字典中添加新键时可能会改变已有键的顺序

- 往字典中添加新键时， Python 解释器可能做出为字典扩容的决定
  - 扩容会导致新建一个更大的散列表，并把字典中已有的元素添加到新表中
    - 添加的过程中可能会发生新的散列冲突，导致新散列表中键的次序发生变化
- 因此，不要对字典同时进行迭代和修改
  - 因为在 Python3 中， `.keys()`, `.items()`， 和 `.values()` 方法返回的都是字典视图
  - 视图具有动态特性，能实时反馈字典的变化   

### `set` 的实现以及导致的结果

- `set` 和 `frozenset` 的实现也依赖散列表，其在散列表中存放的只有元素的引用（相当于在字典中只存放键
而没有值）  

###### 集合的特点

1. 集合中的元素必须是可散列的
2. 集合很消耗内存
3. 可以很高效的判断元素是否存在于某个集合
4. 元素的次序取决于被添加到集合中的次序
5. 往集合中添加元素，可能会改变集合中已有元素的次序